In [2]:
import os
#need to load this due to environment variables not functioning
from dotenv import load_dotenv
import requests
import pandas as pd
load_dotenv('D:/Documents/GitHub/Statistical-Modelling-Project/notebooks/.env')

True

In [3]:
YELP_API_KEY = os.getenv("YELP_API_KEY")
category = 'restaurants'
yelp_url = "https://api.yelp.com/v3/businesses/search"
latitude = None
longitude = None


In [4]:
# Load dataframe from citybikes to extract latitude and longitude. Will loop using all
# latitude and longitude values from this file.
montreal_bike_data = pd.read_csv('montreal_df.csv')


In [5]:
params_yelp = {
    'term': 'restaurant',
    'latitude': latitude,
    'longitude': longitude,
    'radius': 1000,  
    'limit': 50
}

headers_yelp = {
     #"Accept": "application/json", #This caused an error when integrated into code
    'Authorization': f'Bearer {YELP_API_KEY}'
}

In [6]:
try:
    stored_business_list = []

#loop through montreal_bike_data to get latitude and longitude
    for index, row in montreal_bike_data.iterrows():
        latitude = row['latitude']
        longitude = row['longitude']

#Update the parameters with extracted latitude and longitude from montreal_bike_data.
        params_yelp['latitude'] = latitude
        params_yelp['longitude'] = longitude

# Make the API request using the data extracted from above
        response_yelp = requests.get(yelp_url, params=params_yelp, headers=headers_yelp)
        response_yelp.raise_for_status()

#Check for successful response and store JSON
        if response_yelp.status_code == 200:
            result_data_yelp = response_yelp.json()
            businesses = result_data_yelp.get('businesses', [])

            for business in businesses:
                business_info = {
                    'Name': business['name'],
                    'Rating': business['rating'],
                    'Address': ', '.join(business['location']['display_address']),
                    'Latitude': latitude,
                    'Longitude': longitude
                }
                stored_business_list.append(business_info)
        else:
            print(f"Yelp request failed for latitude {latitude}, longitude {longitude}. Error: {response_yelp.status_code}")

except requests.exceptions.RequestException as e:
    print(f"Yelp request failed. Error: {e}")

# Create DataFrame from all_businesses list of dictionaries
yelp_df = pd.DataFrame(stored_business_list)


print(yelp_df)            
yelp_df.to_csv('yelp_data.csv', index=False)

Yelp request failed. Error: 429 Client Error: Too Many Requests for url: https://api.yelp.com/v3/businesses/search?term=restaurant&latitude=45.51694782364431&longitude=-73.62631559371948&radius=1000&limit=50
                               Name  Rating  \
0                              LAON     4.8   
1                            O'Thym     4.5   
2                   Les Deux Gamins     4.2   
3                            Sparta     4.7   
4                      Poutineville     4.3   
...                             ...     ...   
14480                           PFK     2.5   
14481  Bofinger Barbecue Smokehouse     2.9   
14482               Trattoria Mario     0.0   
14483                    Sushi Shop     2.5   
14484           Restaurant Copoli-2     3.9   

                                                 Address   Latitude  Longitude  
0      1657 Rue Saint-Denis, Montreal, QC H2X 3K4, Ca...  45.516926 -73.564257  
1      1112 Boulevard de Maisonneuve E, Montreal, QC ...  45.5169

In [ ]:
#imports
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import time

#used dotenv to store API key like an environment variable. Was unable to create environment variable successfully in CMD or Bash. 
load_dotenv('D:/Documents/GitHub/Statistical-Modelling-Project/notebooks/.env')

#set the API key to a variable for use in request. 
YELP_API_KEY = os.getenv("YELP_API_KEY")
category = 'restaurants'
yelp_url = "https://api.yelp.com/v3/businesses/search"

# Load dataframe from citybikes to extract latitude and longitude. Will loop using all
# latitude and longitude values from this file.
montreal_bike_data = pd.read_csv('montreal_df.csv')

#set parameters for request
params_yelp = {
    'term': 'restaurant',
    'latitude': None,
    'longitude': None,
    'radius': 1000,  
    'limit': 50
}

headers_yelp = {
    'Authorization': f'Bearer {YELP_API_KEY}'
}

#create business list. Will be storing businesses from iterated parameters 
stored_business_list = []

#counting the number of requests made to avoid 429 error. 
requests_made = 0
max_requests_per_day = 300

#create a function to make API request
def make_yelp_api_request(params):
    try:
        response_yelp = requests.get(yelp_url, params=params, headers=headers_yelp)
        response_yelp.raise_for_status()
        return response_yelp.json(), response_yelp.status_code
    
    # return string if request is not successful
    except requests.exceptions.RequestException as e:
        print(f"Yelp request failed. Error: {e}")
        return None, 0

#deal with potential exceptions that may occur during loop
try:
    # Loop through montreal_bike_data to get latitude and longitude
    for index, row in montreal_bike_data.iterrows():
        if requests_made >= max_requests_per_day:
            print(f"Reached maximum requests per day ({max_requests_per_day}). Stopping further requests.")
            break

        latitude = row['latitude']
        longitude = row['longitude']

        # Update the parameters with extracted latitude and longitude from montreal_bike_data.
        params_yelp['latitude'] = latitude
        params_yelp['longitude'] = longitude

        result_data_yelp, status_code = make_yelp_api_request(params_yelp)
        requests_made += 1

        if status_code == 200:
            businesses = result_data_yelp.get('businesses', [])

            for business in businesses:
                business_info = {
                    'Name': business['name'],
                    'Rating': business['rating'],
                    'Address': ', '.join(business['location']['display_address']),
                    'Latitude': latitude,
                    'Longitude': longitude
                }
                stored_business_list.append(business_info)

        elif status_code == 429:
            print(f"Rate limit exceeded. Stopping further requests for today.")
            break
        else:
            print(f"Yelp request failed for latitude {latitude}, longitude {longitude}. Error: {status_code}")

except Exception as e:
    print(f"Exception occurred: {e}")

# Create DataFrame from stored_business_list of dictionaries
yelp_df = pd.DataFrame(stored_business_list)

# Output the DataFrame
print(yelp_df)
yelp_df.to_csv('yelp_data.csv', index=False)

In [1]:
#ADDING LATITUDE AND LONGITUDE THAT WERE REFERENCED IN MONTREAL_DF. WAS NOT CORRECTLY MERGED IN PART 2
#THIS IS THE REWORKED CODE

import os
from dotenv import load_dotenv
import requests
import pandas as pd

# Load environment variables
load_dotenv('D:/Documents/GitHub/Statistical-Modelling-Project/notebooks/.env')

# Set the Yelp API key
YELP_API_KEY = os.getenv("YELP_API_KEY")
category = 'restaurants'
yelp_url = "https://api.yelp.com/v3/businesses/search"

# Load dataframe from montreal_df.csv to extract latitude and longitude
montreal_bike_data = pd.read_csv('montreal_df.csv')

# Set parameters for Yelp API request
params_yelp = {
    'term': 'restaurant',
    'latitude': None,
    'longitude': None,
    'radius': 1000,  
    'limit': 50
}

# Headers for Yelp API request
headers_yelp = {
    'Authorization': f'Bearer {YELP_API_KEY}'
}

# List to store extracted businesses
stored_business_list = []

# Function to make Yelp API request
def make_yelp_api_request(params):
    try:
        response_yelp = requests.get(yelp_url, params=params, headers=headers_yelp)
        response_yelp.raise_for_status()
        return response_yelp.json(), response_yelp.status_code
    except requests.exceptions.RequestException as e:
        print(f"Yelp request failed. Error: {e}")
        return None, 0

try:
    # Loop through montreal_bike_data to get latitude and longitude
    for index, row in montreal_bike_data.iterrows():
        latitude = row['latitude']
        longitude = row['longitude']

        # Update parameters with latitude and longitude from montreal_bike_data
        params_yelp['latitude'] = latitude
        params_yelp['longitude'] = longitude

        # Make Yelp API request
        result_data_yelp, status_code = make_yelp_api_request(params_yelp)

        if status_code == 200:
            businesses = result_data_yelp.get('businesses', [])

            for business in businesses:
                business_info = {
                    'Name': business['name'],
                    'Rating': business['rating'],
                    'Address': ', '.join(business['location']['display_address']),
                    'OriginalLatitude': latitude,  # Store original latitude from montreal_df
                    'OriginalLongitude': longitude,  # Store original longitude from montreal_df
                    'BusinessLatitude': business['coordinates']['latitude'],  # Latitude of the business from Yelp
                    'BusinessLongitude': business['coordinates']['longitude']  # Longitude of the business from Yelp
                }
                stored_business_list.append(business_info)

        elif status_code == 429:
            print(f"Rate limit exceeded. Stopping further requests for today.")
            break
        else:
            print(f"Yelp request failed for latitude {latitude}, longitude {longitude}. Error: {status_code}")

except Exception as e:
    print(f"Exception occurred: {e}")

# Create DataFrame from stored_business_list of dictionaries
yelp_df = pd.DataFrame(stored_business_list)

# Output the DataFrame
print(yelp_df)
yelp_df.to_csv('yelp_data.csv', index=False)

Yelp request failed. Error: 429 Client Error: Too Many Requests for url: https://api.yelp.com/v3/businesses/search?term=restaurant&latitude=45.491513&longitude=-73.633649&radius=1000&limit=50
Yelp request failed for latitude 45.491513, longitude -73.633649. Error: 0
Yelp request failed. Error: 429 Client Error: Too Many Requests for url: https://api.yelp.com/v3/businesses/search?term=restaurant&latitude=45.46892698487744&longitude=-73.6198702454567&radius=1000&limit=50
Yelp request failed for latitude 45.46892698487744, longitude -73.6198702454567. Error: 0
Yelp request failed. Error: 429 Client Error: Too Many Requests for url: https://api.yelp.com/v3/businesses/search?term=restaurant&latitude=45.47710675189806&longitude=-73.6214379966259&radius=1000&limit=50
Yelp request failed for latitude 45.47710675189806, longitude -73.6214379966259. Error: 0
Yelp request failed. Error: 429 Client Error: Too Many Requests for url: https://api.yelp.com/v3/businesses/search?term=restaurant&latitude